In [1]:
#Mocap Arena Camera Calibrate Program
#Mateus Pincho - calibrate intrisics using OpenCV
from glob import glob
import cv2
import matplotlib.pyplot as plt
import numpy as np

In [2]:
# Funções de calibração

#Reconstrução do tabuleiro
def construct3DPoints(patternSize,squareSize):
    X = np.zeros((patternSize[0]*patternSize[1],3), np.float32)
    X[:,:2] = np.mgrid[0:patternSize[0],0:patternSize[1]].T.reshape(-1,2)
    X = X * squareSize # Square size não interfere na calibração -> somente um fator de escala
                       # o square size não é utilizado na calibração dos cara de stanford
    return X

#Detectando os corners
def detectCorners(images, boardPoints, patternSize):
    worldPoints = []
    imagePoints = [] 

    img_size = 0
    counter = 0
    for fname in images:
        img = cv2.imread(fname)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        img_size = gray.shape[::-1]
        ret, corners = cv2.findChessboardCornersSB(gray, patternSize, None)
        if ret == True:
            #print("Corners found in image " + str(fname)) #- see if corners are found 
            #corners2 = cv2.cornerSubPix(gray, corners, (11,11), (-1,-1), criteria=(cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001))
            worldPoints.append(boardPoints)
            imagePoints.append(corners)
            counter+=1

    print("Corners found in " + str(counter) + " images")
    #print(img_size)
    return worldPoints, imagePoints, img_size

#Descobrindo os intrisicos, extrinsecos e o erro de reprojeção 
def calibrate(images, useFisheye, patternSize, squareSize):

    boardPoints = construct3DPoints(patternSize, squareSize)

    worldPoints, imagePoints, imgSize = detectCorners(images, boardPoints, patternSize)

    if useFisheye:
        flagsCalib = cv2.fisheye.CALIB_RECOMPUTE_EXTRINSIC+cv2.fisheye.CALIB_FIX_SKEW+cv2.fisheye.CALIB_CHECK_COND
        calibrateCriteria = (cv2.TERM_CRITERIA_EPS+cv2.TERM_CRITERIA_MAX_ITER,30, 1e-12)

        ret, cameraMatrix, k, R, t = cv2.fisheye.calibrate(np.expand_dims(np.asarray(worldPoints), -2), imagePoints, imgSize, None, None,
                                                                    flags=flagsCalib,criteria=calibrateCriteria)
    else:
        flagsCalib = cv2.CALIB_RATIONAL_MODEL

        ret, cameraMatrix, k, rvecs, tvecs, stdIntrinc, stdExtrinsic, perViewError = cv2.calibrateCameraExtended(worldPoints, imagePoints, imgSize, None, None,
                                                                flags=flagsCalib)

    print("RMS re-projection error:", ret)
    print("The median re-projection error", np.median(perViewError))
    print("Camera Matrix:\n", cameraMatrix)
    print("Distortion Parameters:\n", k)

    return cameraMatrix, k, rvecs, tvecs, stdIntrinc, stdExtrinsic, perViewError

In [5]:
dataset =  ['../../images/virtual/original/set_5/*.jpg',  # dataset[0]
              '../../images/virtual/original/set_10/*.jpg',  # dataset[1]
              '../../images/virtual/original/set_15/*.jpg',  # dataset[2]
              '../../images/virtual/original/set_20/*.jpg',  # dataset[3]
              '../../images/virtual/original/set_25/*.jpg',  # dataset[4]
              '../../../images/virtual/distorted/set_10/*.jpg',  # dataset[5]
              '../../images/chess/opencv/*.jpg']    # dataset[6]

In [6]:
# escolha o conjunto de imagens
image_set = glob('../../../images/virtual/7x7/distorted/set_25/*.jpg')

print(image_set)

# defina os parâmetros da calibração
pattern_size = (7,7)
square_size = 30
useFisheye = False

['../../../images/virtual/7x7/distorted/set_25\\image 0.jpg', '../../../images/virtual/7x7/distorted/set_25\\image 1.jpg', '../../../images/virtual/7x7/distorted/set_25\\image 10.jpg', '../../../images/virtual/7x7/distorted/set_25\\image 11.jpg', '../../../images/virtual/7x7/distorted/set_25\\image 12.jpg', '../../../images/virtual/7x7/distorted/set_25\\image 13.jpg', '../../../images/virtual/7x7/distorted/set_25\\image 14.jpg', '../../../images/virtual/7x7/distorted/set_25\\image 15.jpg', '../../../images/virtual/7x7/distorted/set_25\\image 16.jpg', '../../../images/virtual/7x7/distorted/set_25\\image 17.jpg', '../../../images/virtual/7x7/distorted/set_25\\image 18.jpg', '../../../images/virtual/7x7/distorted/set_25\\image 19.jpg', '../../../images/virtual/7x7/distorted/set_25\\image 2.jpg', '../../../images/virtual/7x7/distorted/set_25\\image 20.jpg', '../../../images/virtual/7x7/distorted/set_25\\image 21.jpg', '../../../images/virtual/7x7/distorted/set_25\\image 22.jpg', '../../../

In [8]:
camera_matrix, distortion, rotation, translation, stdInt, stdExt, _ = calibrate(image_set, useFisheye, pattern_size, square_size)

Corners found in 25 images
RMS re-projection error: 0.4163958724772668
The median re-projection error 0.3882963658734905
Camera Matrix:
 [[402.31631666   0.         239.5468603 ]
 [  0.         402.260159   238.94865854]
 [  0.           0.           1.        ]]
Distortion Parameters:
 [[ 4.82005659e+01  1.29362298e+02 -3.16916596e-04  8.44455180e-05
  -3.25269515e+02  4.58327540e+01  1.38609375e+02 -3.27680911e+02
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00]]


In [12]:
stdInt

array([[1.79647836e+00],
       [1.78381942e+00],
       [5.94267120e-01],
       [6.07506855e-01],
       [1.71688329e+01],
       [2.70337586e+02],
       [4.02313272e-04],
       [4.12440439e-04],
       [1.22409372e+02],
       [1.64264023e+01],
       [2.60180052e+02],
       [1.03664662e+02],
       [0.00000000e+00],
       [0.00000000e+00],
       [0.00000000e+00],
       [0.00000000e+00],
       [0.00000000e+00],
       [0.00000000e+00]])

## Calculate the reprojection errors 

---

In [17]:
def calculate_mean_error(image_set, pattern_size, square_size):
    
    # Create empty arrays for rotation and translation vectors
    rvecs = []
    tvecs = []
    errors = []
    
    # Define the world board points
    board_points = construct3DPoints(pattern_size, square_size)
    board_points = board_points.astype('float32')
    
    # Calculate image points in each image
    for image in image_set:
        img = cv2.imread(image)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        
        # Detect chessboard corners in the image
        ret, detected_image_points = cv2.findChessboardCornersSB(gray, pattern_size, None)
        
        # If it was detected
        if ret == True: 

            # Calculate extrinsic parameters 
            _, rvec, tvec = cv2.solvePnP(board_points, detected_image_points, camera_matrix, distortion)
            rvecs.append(rvec)
            tvecs.append(tvecs)

            # Calculate projected image points
            projected_image_points, _ = cv2.projectPoints(board_points, rvec, tvec, camera_matrix, distortion)

            # Find the Euclidean Distance between projected and detected image points
            error = cv2.norm(detected_image_points, projected_image_points, normType= cv2.NORM_L2) / len(projected_image_points)
            
            errors.append(error)
            
            print(f"Mean Error of Image {image}:", error, "px")
    
    mean_error = np.median(errors)
    
    print("Median error: ", mean_error)
    
    return mean_error

def calculate_rms_error(image_set, pattern_size, square_size):
    
    # Create empty arrays for rotation and translation vectors
    rvecs = []
    tvecs = []
    errors = []

    # Define the world board points
    board_points = construct3DPoints(pattern_size, square_size)

    # Calculate the detected image points
    for image in image_set:
        img = cv2.imread(image)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        
        # Detect chessboard corners in the image
        ret, corners = cv2.findChessboardCornersSB(gray, pattern_size, None)

        # If it was detected
        if ret == True:
            
            # Calculate extrinsic parameters
            _, rvec, tvec = cv2.solvePnP(board_points, corners, camera_matrix, distortion)
            rvecs.append(rvec)
            tvecs.append(tvecs)

            # Calculate projected image points
            image_points, _ = cv2.projectPoints(board_points, rvec, tvec, camera_matrix, distortion)

            # Find the Euclidean Distance between projected and detected image points
            error = cv2.norm(corners, image_points, normType= cv2.NORM_L2) / sqrt(len(image_points))
            errors.append(error)

            print(f"RMS Error of Image {image}:", error, "px")

    rms_error = np.median(errors)

    print("Median RMS error: ", rms_error)

    return rms_error

In [19]:
median_error = calculate_mean_error(image_set, pattern_size, square_size)

Mean Error of Image ../../../images/virtual/7x7/distorted/set_25\image 0.jpg: 0.052257617474948836 px
Mean Error of Image ../../../images/virtual/7x7/distorted/set_25\image 1.jpg: 0.05463311137446303 px
Mean Error of Image ../../../images/virtual/7x7/distorted/set_25\image 10.jpg: 0.07053668168413049 px
Mean Error of Image ../../../images/virtual/7x7/distorted/set_25\image 11.jpg: 0.05152536773517064 px
Mean Error of Image ../../../images/virtual/7x7/distorted/set_25\image 12.jpg: 0.062305230180814804 px
Mean Error of Image ../../../images/virtual/7x7/distorted/set_25\image 13.jpg: 0.05379455115525598 px
Mean Error of Image ../../../images/virtual/7x7/distorted/set_25\image 14.jpg: 0.05812727284612817 px
Mean Error of Image ../../../images/virtual/7x7/distorted/set_25\image 15.jpg: 0.05923955466111585 px
Mean Error of Image ../../../images/virtual/7x7/distorted/set_25\image 16.jpg: 0.07781321532206982 px
Mean Error of Image ../../../images/virtual/7x7/distorted/set_25\image 17.jpg: 0.0

In [20]:
rms_error = calculate_rms_error(image_set, pattern_size, square_size)

RMS Error of Image ../../../images/virtual/7x7/distorted/set_25\image 0.jpg: 0.36580332232464186 px
RMS Error of Image ../../../images/virtual/7x7/distorted/set_25\image 1.jpg: 0.3824317796212412 px
RMS Error of Image ../../../images/virtual/7x7/distorted/set_25\image 10.jpg: 0.4937567717889135 px
RMS Error of Image ../../../images/virtual/7x7/distorted/set_25\image 11.jpg: 0.3606775741461945 px
RMS Error of Image ../../../images/virtual/7x7/distorted/set_25\image 12.jpg: 0.43613661126570363 px
RMS Error of Image ../../../images/virtual/7x7/distorted/set_25\image 13.jpg: 0.3765618580867919 px
RMS Error of Image ../../../images/virtual/7x7/distorted/set_25\image 14.jpg: 0.40689090992289717 px
RMS Error of Image ../../../images/virtual/7x7/distorted/set_25\image 15.jpg: 0.41467688262781094 px
RMS Error of Image ../../../images/virtual/7x7/distorted/set_25\image 16.jpg: 0.5446925072544887 px
RMS Error of Image ../../../images/virtual/7x7/distorted/set_25\image 17.jpg: 0.5446925072544887 p